In [166]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.neighbors import KernelDensity
from sklearn import metrics

In [126]:
leb_df = pd.read_csv('../pro-test/data/leb_cleand.csv')
full_df = pd.read_csv('../pro-test/data/leb_cleand.csv')

In [127]:
full_df.shape

(3224, 38)

In [128]:
full_df['sizeexact'] = full_df['sizeexact'].fillna(0)
full_df['sizeestimate'] = full_df['sizeestimate'].fillna(0)
full_df['combined_sizes'] = full_df['sizeexact'] + full_df['sizeestimate']
full_df.drop(columns=['sizeexact','sizeestimate'], inplace=True)

In [129]:
full_df.shape

(3224, 37)

In [130]:
mean_size_imputer = SimpleImputer(missing_values=-99,strategy="mean")
full_df['combined_sizes'] = mean_size_imputer.fit_transform(full_df[['combined_sizes']])

In [131]:
full_df.columns

Index(['recordnumber', 'startdate', 'ongoing', 'governorate', 'district',
       'location', 'gpslatend', 'gpslongend', 'locationtypeend',
       'demandtypeone', 'demandtypetwo', 'demandother', 'orgtypefirst',
       'tacticprimary', 'tacticsecondary', 'violence', 'propertydamage',
       'repression', 'teargas', 'rubberbullets', 'liveammo', 'sticks',
       'surround', 'dogs', 'cleararea', 'policerepress', 'milrepress',
       'militiarepress', 'partyrepress', 'militia or party', 'employerrepress',
       'otherrepress', 'killed', 'injured', 'arrested', 'country',
       'combined_sizes'],
      dtype='object')

In [132]:
# combined_sizes = full_df[['combined_sizes']]

In [133]:
data = full_df.drop(columns=['orgtypefirst','ongoing','startdate','recordnumber','gpslatend','gpslongend','location','district','demandtypetwo','demandother','tacticsecondary','propertydamage',
       'teargas', 'rubberbullets', 'liveammo', 'sticks',
       'surround', 'dogs', 'cleararea', 'policerepress', 'milrepress',
       'militiarepress', 'partyrepress', 'militia or party', 'employerrepress',
       'otherrepress', 'killed', 'injured', 'arrested', 'combined_sizes'])

In [147]:
data.columns

Index(['governorate', 'locationtypeend', 'demandtypeone', 'tacticprimary',
       'violence', 'repression', 'country'],
      dtype='object')

In [134]:
X = data.drop(columns='repression')
y = data[['repression']]

In [135]:
ohe = OneHotEncoder()
ohe.fit_transform(X)

In [137]:
X = X.toarray()

In [138]:
oen = OrdinalEncoder()
y = oen.fit_transform(y)

In [139]:
# y = pd.get_dummies(full_df['repression'])
# y.columns = y.columns.str.replace(' ', '_')
# y.columns = y.columns.str.replace('/', '')
# y.columns = y.columns.str.replace(',', '')

In [140]:
# data = pd.concat([X,y],axis=1)
# data.columns

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [142]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
kde.fit(X_train)
kde.score_samples(X)

array([42.91103265, 42.16381824, 39.96659367, ..., 39.96659367,
       41.06520595, 17.36448894])

In [145]:
kde.score(X_test, y_test)

40296.18621667611

In [144]:
clf = DecisionTreeClassifier(random_state=2)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2, splitter='best')

In [146]:
clf.score(X_test, y_test)

0.7744360902255639

In [164]:
# X_test_new = pd.DataFrame({'governorate':['Baghdad'], 'locationtypeend':['Midan'], 'demandtypeone':['Politics (national)'],
#                            'tacticprimary':['Demonstration / protest'],'violence':['Riot'],'country':['Iraq']})
# X_test_new

In [167]:
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.7744360902255639

In [78]:
# scores = cross_val_score(estimator=clf, X=X_train, y=y_train['Injuries_inflicted'], cv=10)